In [71]:
# imports 
import numpy as np
import pandas as pd

In [92]:
df= pd.read_csv('../data/train/diamonds_train.csv', index_col=[0])
df.head().T

,0,1,2,3,4
index_id,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,41667f6e2629360aecaf00b20f8732e3310417ebd54b24...,01f8667f50d52677bea23231a74156e4f92360d7bc3db6...,c3867352aab641358faec75d733af012dbe2259a014ea8...,0da4b104c4d8589fcb96a03aa0787549a2631935b0f499...
depth,62.4,61.6,62.3,59.6,60.2
table,58.0,58.0,58.0,60.0,62.0
x,6.83,6.4,5.86,7.58,5.4
y,6.79,6.35,5.8,7.48,5.33
z,4.25,3.93,3.63,4.49,3.23
price,4268,3513,1792,7553,1176
carat,1.21,1.02,0.77,1.51,0.57
cut,Premium,Premium,Premium,Premium,Premium
color,J,J,J,J,J


In [93]:
df.drop(labels='index_id', axis=1, inplace= True)
target_name = "price"
target = df[target_name]

diamonds = df.drop(columns=[target_name])
diamonds

,depth,table,x,y,z,carat,cut,color,clarity,city
0,62.4,58.0,6.83,6.79,4.25,1.21,Premium,J,VS2,Dubai
1,61.6,58.0,6.40,6.35,3.93,1.02,Premium,J,VS2,Dubai
2,62.3,58.0,5.86,5.80,3.63,0.77,Premium,J,VS2,Dubai
3,59.6,60.0,7.58,7.48,4.49,1.51,Premium,J,VS2,Dubai
4,60.2,62.0,5.40,5.33,3.23,0.57,Premium,J,VS2,Dubai
...,...,...,...,...,...,...,...,...,...,...
40450,62.2,54.0,5.24,5.27,3.27,0.54,Ideal,F,IF,Surat
40451,61.9,54.0,5.22,5.25,3.24,0.53,Ideal,F,IF,Surat
40452,62.3,55.0,4.30,4.34,2.69,0.30,Ideal,F,IF,Surat
40453,60.9,55.0,4.15,4.23,2.55,0.26,Ideal,F,IF,Surat


In [101]:
numerical_columns = ['carat', 'depth', 'table', 'x', 'y', 'z']
categorical_columns = ['cut', 'color', 'clarity']
FEATS = numerical_columns + categorical_columns
TARGET = 'price'


In [102]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder



categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [103]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])



In [104]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_columns),
                                                ('cat', categorical_transformer, categorical_columns)])

In [105]:
pd.DataFrame(data=preprocessor.fit_transform(df)).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.867006,0.452019,0.247981,0.978807,0.921985,1.022657,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.467458,-0.106755,0.247981,0.596394,0.538254,0.563582,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.058262,0.382172,0.247981,0.116154,0.058590,0.133198,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.497870,-1.503688,1.143433,1.645806,1.523746,1.366964,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.478838,-1.084608,2.038886,-0.292939,-0.351305,-0.440646,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
diamonds_train, diamonds_test = train_test_split(diamonds)

In [126]:
diamonds

,depth,table,x,y,z,carat,cut,color,clarity,city
0,62.4,58.0,6.83,6.79,4.25,1.21,Premium,J,VS2,Dubai
1,61.6,58.0,6.40,6.35,3.93,1.02,Premium,J,VS2,Dubai
2,62.3,58.0,5.86,5.80,3.63,0.77,Premium,J,VS2,Dubai
3,59.6,60.0,7.58,7.48,4.49,1.51,Premium,J,VS2,Dubai
4,60.2,62.0,5.40,5.33,3.23,0.57,Premium,J,VS2,Dubai
...,...,...,...,...,...,...,...,...,...,...
40450,62.2,54.0,5.24,5.27,3.27,0.54,Ideal,F,IF,Surat
40451,61.9,54.0,5.22,5.25,3.24,0.53,Ideal,F,IF,Surat
40452,62.3,55.0,4.30,4.34,2.69,0.30,Ideal,F,IF,Surat
40453,60.9,55.0,4.15,4.23,2.55,0.26,Ideal,F,IF,Surat


In [112]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    diamonds, target, random_state=42)

In [116]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', RandomForestRegressor())])

In [117]:
model.fit(data_train, target_train);

In [118]:
y_pred = model.predict(data_test)
y_pred

array([8250.68,  489.11, 8835.36, ..., 1342.42, 5771.2 ,  557.64])

In [119]:
rmse = mean_squared_error(target_test, y_pred)**0.5
rmse

592.5594440925869

In [120]:
#Model prediction Test Dataframe (aka real prediction):
diamonds_train=pd.read_csv('../data/test/diamonds_test.csv', index_col=[0])
diamonds_train

,carat,cut,color,clarity,depth,table,x,y,z,city
id,,,,,,,,,,
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv


In [121]:
#We must order the columns as per the train dataframe columns order:
diamonds_train=diamonds_train[['depth', 'table', 'x', 'y', 'z', 'carat', 'cut', 'color',
       'clarity','city']]

In [122]:
scaler = StandardScaler()
diamonds_train[['depth', 'table', 'x', 'y', 'z', 'carat']]= scaler.fit_transform(diamonds_train[['depth', 'table', 'x', 'y', 'z', 'carat']])
diamonds_train

,depth,table,x,y,z,carat,cut,color,clarity,city
id,,,,,,,,,,
0,0.669500,1.121874,0.075022,0.133236,0.173091,-0.018412,Very Good,F,SI1,Amsterdam
1,-0.514957,-0.219192,0.964007,1.019395,0.870787,0.855078,Ideal,J,VS1,Surat
2,0.321131,1.568896,1.475847,1.400444,1.404319,1.643349,Premium,H,SI1,Kimberly
3,1.435914,-1.560258,0.317472,0.345914,0.487738,0.215939,Very Good,F,SI1,Kimberly
4,0.808848,0.227830,-0.616411,-0.575691,-0.483564,-0.636246,Very Good,F,VS1,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
13480,0.112109,-0.666214,-0.347022,-0.371875,-0.333080,-0.487114,Ideal,E,SI1,Amsterdam
13481,0.321131,-1.113236,-0.023755,-0.008549,0.022608,-0.188849,Ideal,I,VS2,New York City
13482,-0.096913,-1.113236,0.012164,-0.026273,-0.018433,-0.210153,Ideal,F,VS1,Tel Aviv


In [123]:
diamonds_train_encoded= pd.get_dummies(diamonds_train, 
                                    columns=['cut', 'color','clarity'], 
                                    drop_first=True)
diamonds_train_encoded.drop(labels='city', axis=1, inplace=True)
diamonds_train_encoded

,depth,table,x,y,z,carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,0.669500,1.121874,0.075022,0.133236,0.173091,-0.018412,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,-0.514957,-0.219192,0.964007,1.019395,0.870787,0.855078,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0.321131,1.568896,1.475847,1.400444,1.404319,1.643349,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,1.435914,-1.560258,0.317472,0.345914,0.487738,0.215939,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0.808848,0.227830,-0.616411,-0.575691,-0.483564,-0.636246,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.112109,-0.666214,-0.347022,-0.371875,-0.333080,-0.487114,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
13481,0.321131,-1.113236,-0.023755,-0.008549,0.022608,-0.188849,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
13482,-0.096913,-1.113236,0.012164,-0.026273,-0.018433,-0.210153,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [124]:
#model.fit(X,y) #Volvemos a entrenar el modelo con el datframe original (sin la separacion de train y test), para tener mas datos en el entreno

In [125]:
y_pred = model.predict(diamonds_train_encoded)
y_pred


KeyError: "None of [Index(['cut', 'color', 'clarity'], dtype='object')] are in the [columns]"

In [288]:
"""y_pred=scalery.inverse_transform(y_pred.reshape(-1, 1))
y_pred
"""

'y_pred=scalery.inverse_transform(y_pred.reshape(-1, 1))\ny_pred\n'

In [289]:
y_pred=pd.DataFrame(y_pred, columns=['price'])
y_pred.reset_index(inplace=True)
y_pred.rename(columns={"index": "id"}, inplace= True)
y_pred

,id,price
0,0,3578.012527
1,1,6303.365413
2,2,10069.736381
3,3,4639.656407
4,4,2072.741812
...,...,...
13480,13480,1949.865948
13481,13481,2549.745201
13482,13482,3885.969909
13483,13483,1998.645583


In [290]:
y_pred.to_csv('../data/prediction/price_prediction.csv', index= False)